In [1]:
import numpy 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# Importing libraries for building the neural network
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

from sklearn.utils import shuffle
from keras.optimizers import SGD
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix


from numpy import array
from numpy import argmax
from keras.utils import to_categorical
from sklearn.model_selection import StratifiedKFold

Using TensorFlow backend.


In [2]:
excel_data = pd.read_excel('data/oczyszczone.xls', sheet_name = 'daneSOB')
headers = excel_data.columns.values
excel_data.head()

,Płeć,Wiek,Lokalizacja bólu na początku zachorowania,Lokalizacja bólu obecnie,Intensywność bólu,Czynniki nasilające ból,Czynniki przynoszące ulgę,Progresja bólu,Czas trwania bólu,Charakter bólu na początku zachorowania,...,Tętno,Ruchy oddechowe powłok brzusznych,Wzdęcia,Umiejscowienie bolesności uciskowej,Objaw Blumberga,Obrona mięśniowa,Wzmożone napięcie powłok brzusznych,Opory patologiczne,Objaw Murphy'ego,Klasy diagnozy
0,2,5,7,7,1,3,2,1,3,2,...,5,2,2,8,2,1,2,2,1,1
1,2,5,4,1,3,3,1,2,3,2,...,7,2,1,3,2,1,1,2,1,1
2,2,3,4,5,2,3,2,3,4,2,...,4,1,2,2,1,2,1,1,1,1
3,2,5,7,2,1,3,1,2,2,2,...,7,2,1,11,2,2,2,1,1,1
4,2,3,4,11,2,3,3,2,3,1,...,4,1,2,6,2,1,2,2,2,1


In [3]:
excel_data.groupby('Klasy diagnozy').size()

Klasy diagnozy
1    141
2     17
3     29
4     28
5     55
6     32
7    157
8     17
dtype: int64

In [4]:
excel_data = shuffle(excel_data)
# one hot encoding
for i in headers:
    le = LabelEncoder()
    le.fit(excel_data[str(i)])
    excel_data[str(i)] = le.transform(excel_data[str(i)])
    encoded = to_categorical(excel_data[str(i)])
    
    if len(encoded[0]) > 2 and i != 'Klasy diagnozy':
        insert_loc = numpy.where(headers==i)
        insert_loc = insert_loc[0][0]

        excel_data = excel_data.drop(columns = i)
        for label in range(len(encoded[0])):
            excel_data.insert(int(insert_loc), str(i) + "_" + str(label), encoded[:,label])
    

excel_data.head()


,Płeć,Wiek_4,Lokalizacja bólu na początku zachorowania_10,Lokalizacja bólu obecnie_10,Intensywność bólu_2,Czynniki nasilające ból_3,Czynniki przynoszące ulgę_2,Progresja bólu_2,Czas trwania bólu_3,Charakter bólu na początku zachorowania_2,...,Poprzednie operacje brzuszne,Leki,Ruchy oddechowe powłok brzusznych,Wzdęcia,Objaw Blumberga,Obrona mięśniowa,Wzmożone napięcie powłok brzusznych,Opory patologiczne,Objaw Murphy'ego,Klasy diagnozy
54,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,0,1,0,0,0,1,0,0,0
311,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0,0,0,0,0,0,1,1,0,6
143,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0,0,1,1,0,0,0,0,0,1
314,0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0,1,0,1,0,0,1,0,0,6
428,0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,...,0,1,0,0,0,0,1,1,1,6


In [5]:
# sc = StandardScaler() 
# data = sc.fit_transform(excel_data)
# scaled_df = pd.DataFrame(excel_data, columns=headers)

# for i in headers:
#     excel_data[str(i)]= scaled_df[str(i)]
# data[' Age '], data['Alkphos'], data['sgpt'], data['sgot'],data['TP'],data['ALB'],data['TB'],data['DB'] = dt[' Age '], dt['Alkphos'], dt['sgpt'], dt['sgot'] ,dt['TP'],dt['ALB'],dt['TB'],dt['DB']
#data[' Age '] = dt[' Age ']

In [6]:
# X_data set with arguments
# Y_data set with diagnostick class

Y_data = excel_data['Klasy diagnozy']
X_data = excel_data.drop(columns = 'Klasy diagnozy')

In [7]:
X_data.head()

,Płeć,Wiek_4,Lokalizacja bólu na początku zachorowania_10,Lokalizacja bólu obecnie_10,Intensywność bólu_2,Czynniki nasilające ból_3,Czynniki przynoszące ulgę_2,Progresja bólu_2,Czas trwania bólu_3,Charakter bólu na początku zachorowania_2,...,Żółtaczka w przeszłości,Poprzednie operacje brzuszne,Leki,Ruchy oddechowe powłok brzusznych,Wzdęcia,Objaw Blumberga,Obrona mięśniowa,Wzmożone napięcie powłok brzusznych,Opory patologiczne,Objaw Murphy'ego
54,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,0,1,0,0,0,1,0,0
311,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,1,1,0
143,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0,0,0,1,1,0,0,0,0,0
314,0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0,0,1,0,1,0,0,1,0,0
428,0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,...,0,0,1,0,0,0,0,1,1,1


In [8]:
# Zastanowić się nad dzieleniem zbioru wg klas diagnozy
# def shuffle_by_diagnostic_class(excel_data):
#    pass
  
# shuffle_by_diagnostic_class(excel_data)    

In [9]:
# ratio 1/2 to 1/2
X_train = X_data[:238]
Y_train = Y_data[:238]

X_test = X_data[238:]
Y_test = Y_data[238:]

In [10]:
model = Sequential()
model.add(Dense(units = 32,activation = 'relu',input_dim = 106)) # warstwa wejściowa
model.add(Dense(units = 100,activation = 'relu')) # warstwa ukryta
model.add(Dense(units = 106,kernel_initializer= 'normal', activation = 'softmax')) # warstwa wyjściowa

model.compile(optimizer= SGD(lr = 0.07, momentum=0.0),loss= 'sparse_categorical_crossentropy', metrics= ['accuracy'])
model.fit(X_train,Y_train, epochs = 30,shuffle = True, verbose = 2,validation_data=(X_test,Y_test))


Train on 238 samples, validate on 238 samples
Epoch 1/30
 - 0s - loss: 4.1408 - acc: 0.3487 - val_loss: 2.6679 - val_acc: 0.4370
Epoch 2/30
 - 0s - loss: 2.1776 - acc: 0.4664 - val_loss: 1.8676 - val_acc: 0.3445
Epoch 3/30
 - 0s - loss: 1.5888 - acc: 0.5042 - val_loss: 1.3782 - val_acc: 0.5924
Epoch 4/30
 - 0s - loss: 1.2591 - acc: 0.6471 - val_loss: 1.3657 - val_acc: 0.5672
Epoch 5/30
 - 0s - loss: 1.0703 - acc: 0.6681 - val_loss: 1.0976 - val_acc: 0.6092
Epoch 6/30
 - 0s - loss: 0.8984 - acc: 0.7227 - val_loss: 0.8360 - val_acc: 0.7983
Epoch 7/30
 - 0s - loss: 0.6720 - acc: 0.8445 - val_loss: 0.7919 - val_acc: 0.7563
Epoch 8/30
 - 0s - loss: 0.5921 - acc: 0.8277 - val_loss: 0.5616 - val_acc: 0.8319
Epoch 9/30
 - 0s - loss: 0.4665 - acc: 0.8613 - val_loss: 0.4704 - val_acc: 0.8824
Epoch 10/30
 - 0s - loss: 0.3824 - acc: 0.8992 - val_loss: 0.4015 - val_acc: 0.9034
Epoch 11/30
 - 0s - loss: 0.3204 - acc: 0.9118 - val_loss: 0.3931 - val_acc: 0.9160
Epoch 12/30
 - 0s - loss: 0.2577 - acc:

In [11]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 32)                3424      
_________________________________________________________________
dense_2 (Dense)              (None, 100)               3300      
_________________________________________________________________
dense_3 (Dense)              (None, 106)               10706     
Total params: 17,430
Trainable params: 17,430
Non-trainable params: 0
_________________________________________________________________


In [12]:
y_pred = model.predict(X_test)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_test, (y_pred > 0.1))
print(cm)

ValueError: Classification metrics can't handle a mix of multiclass and multilabel-indicator targets